In [97]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.preprocessing import LabelEncoder
import os
from tqdm import tqdm
import nibabel as nib

In [98]:
data = pd.read_csv('../graph_data_and_gbc_2.0.csv')
data['binary_target'] = data['label'].apply(lambda x: 0 if x == 'CON' else 1)
data = data[['binary_target', 'label','Number of Edges', 'Average Degree', 'Theoretical Avg Clustering', 'Average Clustering',
           'Theoretical Avg Path Length', 'Average Path Length', 'Size of Largest CC','X534','X484', 'X426', 'X284', 'X684']]
data

,binary_target,label,Number of Edges,Average Degree,Theoretical Avg Clustering,Average Clustering,Theoretical Avg Path Length,Average Path Length,Size of Largest CC,X534,X484,X426,X284,X684
0,0,CON,66321,184.738162,0.257654,0.490784,1.260116,1.744172,718,0.002200,0.004631,0.002499,0.004781,0.002086
1,0,CON,67426,187.816156,0.261947,0.520350,1.256139,1.747524,718,0.001221,-0.000033,-0.001197,0.001490,-0.004478
2,1,SADP,62752,174.796657,0.243789,0.412171,1.273615,1.756242,718,0.003515,0.013684,0.019473,0.001079,0.011703
3,0,CON,63635,177.256267,0.247219,0.488305,1.270178,1.755174,718,0.002946,0.001557,0.000913,0.002309,0.007828
4,1,BPP,93362,260.061281,0.362708,0.697369,1.182623,1.685000,717,0.005452,0.014243,0.037410,0.007334,0.006967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,1,SADP,76602,213.376045,0.297596,0.596727,1.226254,1.715637,718,0.012257,0.010740,0.013513,0.002984,-0.010022
634,1,SADP,63932,178.083565,0.248373,0.440672,1.269037,1.751864,718,0.010503,0.005881,-0.005122,-0.005237,0.004355
635,0,CON,92597,257.930362,0.359736,0.659606,1.184376,1.661441,718,0.002872,0.001178,0.002569,0.005938,0.012022
636,0,CON,74396,207.231198,0.289025,0.548060,1.232972,1.716989,718,0.000538,0.002331,-0.003451,0.012152,0.002300


In [99]:
#X_cols = ['Number of Edges', 'Average Degree', 'Theoretical Avg Clustering', 'Average Clustering',
#           'Theoretical Avg Path Length', 'Average Path Length', 'Size of Largest CC','X534','X484', 'X426', 'X284', 'X684']
y_cols = ['binary_target']
X_cols = ['X534','X484', 'X426', 'X284', 'X684']
X = data[X_cols]
y = data[y_cols]
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Binary Classification

In [100]:
seed = 10
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(510, 5) (128, 5) (510, 1) (128, 1)


In [101]:
# Initialize the XGBoost classifier
xgb_clf = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100, seed=seed)

# Fit the model
xgb_clf.fit(X_train, y_train)


# Initialize the Random Forest classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=seed)

# Fit the model
rf_clf.fit(X_train, y_train)

/home/tico/anaconda3/envs/ds_project_env/lib/python3.10/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(random_state=10)

In [102]:
# For XGBoost Model
# Make predictions on training and test sets
xgb_train_predictions = xgb_clf.predict(X_train)
xgb_test_predictions = xgb_clf.predict(X_test)

# Calculate accuracy
xgb_train_accuracy = accuracy_score(y_train, xgb_train_predictions)
xgb_test_accuracy = accuracy_score(y_test, xgb_test_predictions)

# Calculate misclassification rate
xgb_train_misclassification = 1 - xgb_train_accuracy
xgb_test_misclassification = 1 - xgb_test_accuracy

# For Random Forest Model
# Make predictions on training and test sets
rf_train_predictions = rf_clf.predict(X_train)
rf_test_predictions = rf_clf.predict(X_test)

# Calculate accuracy
rf_train_accuracy = accuracy_score(y_train, rf_train_predictions)
rf_test_accuracy = accuracy_score(y_test, rf_test_predictions)

# Calculate misclassification rate
rf_train_misclassification = 1 - rf_train_accuracy
rf_test_misclassification = 1 - rf_test_accuracy

# Print results
print("XGBoost Training Accuracy:", xgb_train_accuracy)
print("XGBoost Test Accuracy:", xgb_test_accuracy)
print("XGBoost Training Misclassification Rate:", xgb_train_misclassification)
print("XGBoost Test Misclassification Rate:", xgb_test_misclassification)

print("Random Forest Training Accuracy:", rf_train_accuracy)
print("Random Forest Test Accuracy:", rf_test_accuracy)
print("Random Forest Training Misclassification Rate:", rf_train_misclassification)
print("Random Forest Test Misclassification Rate:", rf_test_misclassification)

# Calculate the standard error of the misclassification rate for XGBoost
xgb_train_se = np.sqrt((xgb_train_misclassification * (1 - xgb_train_misclassification)) / len(X_train))
xgb_test_se = np.sqrt((xgb_test_misclassification * (1 - xgb_test_misclassification)) / len(X_test))

# Calculate the standard error of the misclassification rate for Random Forest
rf_train_se = np.sqrt((rf_train_misclassification * (1 - rf_train_misclassification)) / len(X_train))
rf_test_se = np.sqrt((rf_test_misclassification * (1 - rf_test_misclassification)) / len(X_test))

# Print the standard errors
print("XGBoost Training Misclassification SE:", xgb_train_se)
print("XGBoost Test Misclassification SE:", xgb_test_se)
print("Random Forest Training Misclassification SE:", rf_train_se)
print("Random Forest Test Misclassification SE:", rf_test_se)

XGBoost Training Accuracy: 1.0
XGBoost Test Accuracy: 0.953125
XGBoost Training Misclassification Rate: 0.0
XGBoost Test Misclassification Rate: 0.046875
Random Forest Training Accuracy: 1.0
Random Forest Test Accuracy: 0.953125
Random Forest Training Misclassification Rate: 0.0
Random Forest Test Misclassification Rate: 0.046875
XGBoost Training Misclassification SE: 0.0
XGBoost Test Misclassification SE: 0.018682740693075187
Random Forest Training Misclassification SE: 0.0
Random Forest Test Misclassification SE: 0.018682740693075187


## Multiclass classification

In [8]:
seed = 10
binary_data = data.drop('binary_target', axis=1)
input_features = binary_data.drop('Group', axis=1).values
# Initialize the LabelEncoder
le = LabelEncoder()
targets_numeric = le.fit_transform(binary_data['Group'])

X_train, X_test, y_train, y_test = train_test_split(input_features, targets_numeric, 
                                                    test_size=0.2, random_state=seed, 
                                                    stratify=targets_numeric)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

# Access the mapping of encoded labels back to original class names
label_mapping = dict(zip(le.transform(le.classes_), le.classes_))
for target_values, name in [(y_train,'y_train'),(y_test,'y_test')]:
    # Get unique values and their counts
    unique_values, counts = np.unique(target_values, return_counts=True)
    print(f'Target Values statistics of: {name}')
    # Display the unique values and their counts
    for value, count in zip(unique_values, counts):
        print(f"Value: {label_mapping[value]}, Count: {count}")

(510, 718) (128, 718) (510,) (128,)
Target Values statistics of: y_train
Value: BPP, Count: 120
Value: CON, Count: 161
Value: SADP, Count: 95
Value: SCZP, Count: 134
Target Values statistics of: y_test
Value: BPP, Count: 30
Value: CON, Count: 41
Value: SADP, Count: 24
Value: SCZP, Count: 33


### XG Boost and Random Forest

In [9]:
# Initialize the XGBoost classifier
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', n_estimators=100, seed=seed)

# Fit the model
xgb_clf.fit(X_train, y_train)


# Initialize the Random Forest classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=seed)

# Fit the model
rf_clf.fit(X_train, y_train)

RandomForestClassifier(random_state=10)

In [10]:
# For XGBoost Model
# Make predictions on training and test sets
xgb_train_predictions = xgb_clf.predict(X_train)
xgb_test_predictions = xgb_clf.predict(X_test)

# Calculate accuracy
xgb_train_accuracy = accuracy_score(y_train, xgb_train_predictions)
xgb_test_accuracy = accuracy_score(y_test, xgb_test_predictions)

# Calculate misclassification rate
xgb_train_misclassification = 1 - xgb_train_accuracy
xgb_test_misclassification = 1 - xgb_test_accuracy

# For Random Forest Model
# Make predictions on training and test sets
rf_train_predictions = rf_clf.predict(X_train)
rf_test_predictions = rf_clf.predict(X_test)

# Calculate accuracy
rf_train_accuracy = accuracy_score(y_train, rf_train_predictions)
rf_test_accuracy = accuracy_score(y_test, rf_test_predictions)

# Calculate misclassification rate
rf_train_misclassification = 1 - rf_train_accuracy
rf_test_misclassification = 1 - rf_test_accuracy

# Print results
print("XGBoost Training Accuracy:", xgb_train_accuracy)
print("XGBoost Test Accuracy:", xgb_test_accuracy)
print("XGBoost Training Misclassification Rate:", xgb_train_misclassification)
print("XGBoost Test Misclassification Rate:", xgb_test_misclassification)

print("Random Forest Training Accuracy:", rf_train_accuracy)
print("Random Forest Test Accuracy:", rf_test_accuracy)
print("Random Forest Training Misclassification Rate:", rf_train_misclassification)
print("Random Forest Test Misclassification Rate:", rf_test_misclassification)

# Calculate the standard error of the misclassification rate for XGBoost
xgb_train_se = np.sqrt((xgb_train_misclassification * (1 - xgb_train_misclassification)) / len(X_train))
xgb_test_se = np.sqrt((xgb_test_misclassification * (1 - xgb_test_misclassification)) / len(X_test))

# Calculate the standard error of the misclassification rate for Random Forest
rf_train_se = np.sqrt((rf_train_misclassification * (1 - rf_train_misclassification)) / len(X_train))
rf_test_se = np.sqrt((rf_test_misclassification * (1 - rf_test_misclassification)) / len(X_test))

# Print the standard errors
print("XGBoost Training Misclassification SE:", xgb_train_se)
print("XGBoost Test Misclassification SE:", xgb_test_se)
print("Random Forest Training Misclassification SE:", rf_train_se)
print("Random Forest Test Misclassification SE:", rf_test_se)

XGBoost Training Accuracy: 1.0
XGBoost Test Accuracy: 0.578125
XGBoost Training Misclassification Rate: 0.0
XGBoost Test Misclassification Rate: 0.421875
Random Forest Training Accuracy: 1.0
Random Forest Test Accuracy: 0.5859375
Random Forest Training Misclassification Rate: 0.0
Random Forest Test Misclassification Rate: 0.4140625
XGBoost Training Misclassification SE: 0.0
XGBoost Test Misclassification SE: 0.04365136062231838
Random Forest Training Misclassification SE: 0.0
Random Forest Test Misclassification SE: 0.043536510010075705


## Data augmentation techniques

In [11]:
#! pip install imbalanced-learn

   ---------------------------------------- 257.7/257.7 kB 1.4 MB/s eta 0:00:00


In [26]:
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [27]:
# Assuming X_train, y_train are your initial training data

# Train a random forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=10)
rf.fit(X_train, y_train)

# Select features based on importance
selector = SelectFromModel(rf, prefit=True, threshold='mean')  # Selects features with importance higher than the mean importance

# Transform training and testing sets
X_train_reduced = selector.transform(X_train)
X_test_reduced = selector.transform(X_test)

print(f"Original number of features: {X_train.shape[1]}")
print(f"Reduced number of features: {X_train_reduced.shape[1]}")

smote = SMOTE(random_state=10)
X_train_smote, y_train_smote = smote.fit_resample(X_train_reduced, y_train)

# Check the distribution after SMOTE
unique_smote, counts_smote = np.unique(y_train_smote, return_counts=True)
for value, count in zip(unique_smote, counts_smote):
    print(f"Value: {label_mapping[value]}, Count: {count}")

# Determine the desired number of samples for each class
desired_counts = 300
target_counts = {label: desired_counts for label in np.unique(y_train_smote)}

# Initialize RandomOverSampler with the target_counts as the sampling strategy
ros = RandomOverSampler(random_state=10, sampling_strategy=target_counts)

# Now, apply RandomOverSampler to adjust class distribution to have 300 samples per class
X_train_augmented, y_train_augmented = ros.fit_resample(X_train_smote, y_train_smote)

# Verify the final distribution
unique_augmented, counts_augmented = np.unique(y_train_augmented, return_counts=True)
print('After augmentation to reach 300 samples per class:')
for value, count in zip(unique_augmented, counts_augmented):
    print(f"Value: {label_mapping[value]}, Count: {count}")

Original number of features: 718
Reduced number of features: 171
Value: BPP, Count: 161
Value: CON, Count: 161
Value: SADP, Count: 161
Value: SCZP, Count: 161
After augmentation to reach 300 samples per class:
Value: BPP, Count: 300
Value: CON, Count: 300
Value: SADP, Count: 300
Value: SCZP, Count: 300


In [28]:
# Initialize the XGBoost classifier
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', n_estimators=100, seed=seed)

# Fit the model
xgb_clf.fit(X_train_augmented, y_train_augmented)


# Initialize the Random Forest classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=seed)

# Fit the model
rf_clf.fit(X_train_augmented, y_train_augmented)

RandomForestClassifier(random_state=10)

In [29]:
# For XGBoost Model
# Make predictions on training and test sets
xgb_train_predictions = xgb_clf.predict(X_train_augmented)
xgb_test_predictions = xgb_clf.predict(X_test_reduced)

# Calculate accuracy
xgb_train_accuracy = accuracy_score(y_train_augmented, xgb_train_predictions)
xgb_test_accuracy = accuracy_score(y_test, xgb_test_predictions)

# Calculate misclassification rate
xgb_train_misclassification = 1 - xgb_train_accuracy
xgb_test_misclassification = 1 - xgb_test_accuracy

# For Random Forest Model
# Make predictions on training and test sets
rf_train_predictions = rf_clf.predict(X_train_augmented)
rf_test_predictions = rf_clf.predict(X_test_reduced)

# Calculate accuracy
rf_train_accuracy = accuracy_score(y_train_augmented, rf_train_predictions)
rf_test_accuracy = accuracy_score(y_test, rf_test_predictions)

# Calculate misclassification rate
rf_train_misclassification = 1 - rf_train_accuracy
rf_test_misclassification = 1 - rf_test_accuracy

# Print results
print("XGBoost Training Accuracy:", xgb_train_accuracy)
print("XGBoost Test Accuracy:", xgb_test_accuracy)
print("XGBoost Training Misclassification Rate:", xgb_train_misclassification)
print("XGBoost Test Misclassification Rate:", xgb_test_misclassification)

print("Random Forest Training Accuracy:", rf_train_accuracy)
print("Random Forest Test Accuracy:", rf_test_accuracy)
print("Random Forest Training Misclassification Rate:", rf_train_misclassification)
print("Random Forest Test Misclassification Rate:", rf_test_misclassification)

# Calculate the standard error of the misclassification rate for XGBoost
xgb_train_se = np.sqrt((xgb_train_misclassification * (1 - xgb_train_misclassification)) / len(X_train))
xgb_test_se = np.sqrt((xgb_test_misclassification * (1 - xgb_test_misclassification)) / len(X_test))

# Calculate the standard error of the misclassification rate for Random Forest
rf_train_se = np.sqrt((rf_train_misclassification * (1 - rf_train_misclassification)) / len(X_train))
rf_test_se = np.sqrt((rf_test_misclassification * (1 - rf_test_misclassification)) / len(X_test))

# Print the standard errors
print("XGBoost Training Misclassification SE:", xgb_train_se)
print("XGBoost Test Misclassification SE:", xgb_test_se)
print("Random Forest Training Misclassification SE:", rf_train_se)
print("Random Forest Test Misclassification SE:", rf_test_se)

XGBoost Training Accuracy: 1.0
XGBoost Test Accuracy: 0.6015625
XGBoost Training Misclassification Rate: 0.0
XGBoost Test Misclassification Rate: 0.3984375
Random Forest Training Accuracy: 1.0
Random Forest Test Accuracy: 0.5859375
Random Forest Training Misclassification Rate: 0.0
Random Forest Test Misclassification Rate: 0.4140625
XGBoost Training Misclassification SE: 0.0
XGBoost Test Misclassification SE: 0.04327284968965728
Random Forest Training Misclassification SE: 0.0
Random Forest Test Misclassification SE: 0.043536510010075705
